# AI-Powered Document Processing and Retrieval

This notebook demonstrates the process of document processing, information extraction, and answering questions using language and vector-based models. Libraries such as `LangChain`, `FAISS`, `Whisper`, and `HuggingFace`

## Extracting podcast from a Podcast Webpage

In [7]:
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore")

# URL strony z podcastem
url = "https://www.99twarzyai.pl/2025/03/25/ai-tinkerers-artur-wala/"

# Pobranie strony
response = requests.get(url, verify = False)
response.raise_for_status()

# Analiza zawartości strony
soup = BeautifulSoup(response.text, "html.parser")

# Znalezienie tagu audio
audio_tag = soup.find("audio")
if audio_tag:
    source_tag = audio_tag.find("source")
    if source_tag and source_tag.get("src"):
        print("Znaleziono źródło audio:", source_tag["src"])

audio_tag

<audio controls="" src="https://api.spreaker.com/v2/episodes/65095132/download.mp3"></audio>

## Transcribing an Audio File

In this section, I use the `Whisper` model to transcribe an audio file into text. The `Whisper` model is an advanced speech processing model that supports multiple languages and works on CPUs, making it convenient for systems with limited hardware resources.

In [15]:
'''
import whisper

whisper_model = whisper.load_model("medium")
file = "podcast.mp3"
transcription = whisper_model.transcribe(file)
transcription
'''

'\nimport whisper\n\nwhisper_model = whisper.load_model("medium")\nfile = "podcast.mp3"\ntranscription = whisper_model.transcribe(file)\ntranscription\n'

## Saving the Transcription to a File

The transcription is saved to a text file so that it can be processed later. We use the `tempfile` module to manage temporary directories.

In [3]:
import tempfile
with tempfile.TemporaryDirectory() as tmpdir:
    transcription = whisper_model.transcribe(file)["text"].strip()

    with open("transcription.txt", "w") as file:
        file.write(transcription)

## Loading the Transcription

We load the saved text file so that it can be processed in the subsequent steps.

In [16]:
with open("transcription.txt" , "r") as file:
    response = file.read()

response[:400]

'W podkaście 99 twarzy AI rozmawiamy o wszystkim, co związane z technologią, innowacjami i oczywiście sztuczną inteligencją. Przygotuj się na luźne, inspirujące i często zaskakujące rozmowy z różnymi ekspertami, programistami, naukowcami, przedsiębiorcami i ludźmi, którzy na co dzień pracują z AI. Podcast 99 twarzy AI. Zaprasza Karl Stryja. Witajcie. Dzisiaj mam ogromną przyjemność rozmawiać z Artu'

##  Using an Embedding Model

I use the `HuggingFaceEmbeddings` model to generate embeddings for the text. Embeddings are vector representations of text that enable efficient searching for similar documents.

In [17]:
from langchain_ollama.llms import OllamaLLM

model = OllamaLLM(model = "SpeakLeash/bielik-11b-v2.3-instruct:Q4_K_M")

In [18]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2",
            model_kwargs={'device': 'cpu'} ) # Use CPU for efficiency)

## Loading Documents

I use `TextLoader` to load text documents that will be processed in the following steps.

In [19]:
from langchain_community.document_loaders import TextLoader

file_path = "transcription.txt"

loader = TextLoader(file_path)
text_document = loader.load()

## Splitting Text into Chunks

I use `RecursiveCharacterTextSplitter` to split the text into smaller chunks. This allows us to efficiently process large documents and search for information within them.

In [20]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50 , length_function=len, add_start_index=True)
documents = text_splitter.split_documents(text_document)

## Creating a Vector Store

Using `FAISS`, I create a vector store that allows for fast searching of similar documents based on user queries.

In [21]:
from langchain_community.vectorstores import FAISS

query = "Co to jest AI tinkers?"

db = FAISS.from_documents(documents , embedding=embeddings_model)
docs = db.similarity_search(query)
print(docs[0].page_content)

że to jest taki aspekt kulturowy. Na następnym AI Tinkers seria porażek? Ciekawy pomysł do rozważenia. Zobaczymy, kiedy będzie kolejny meetup. Nie chcę jeszcze obiecywać kiedy, bo teraz musimy się skupić na hackathonie z OpenAI. To jest jeszcze taki top priority dla nas. Właśnie bardzo płynnie przyszedłeś do tematu, który chciałem poruszyć. Pamiętam naszą rozmowę chyba 2 albo 3 tygodnie temu. Jak ze sobą korespondowaliśmy, pisałeś Karol będzie grubo. No i co, dowiozłem. Gratuluję, Artur.


# Set up Prompt template and Output Parser

In [22]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

prompt = ChatPromptTemplate.from_template(
    """
    Odpowiedz na pytanie wyłącznie w oparciu o poniższy kontekst. Bądź zwięzły.
    Jeśli nie możesz znaleźć odpowiedzi w kontekście, powiedz „Nie mogę odpowiedzieć na to pytanie na podstawie podanego kontekstu”.
        
    Kontekst: {context}
    Pytanie: {question}
    Odpowiedź: """
)

## Creating a Query Processing Chain

I create a query processing chain that combines searching in the vector store with a language model. This allows us to generate answers to user questions based on the context of the documents.

In [23]:
from langchain_core.runnables import RunnablePassthrough

retriver =  db.as_retriever(search_type="similarity")

# def format_docs(docs):
#    return "\n\n".join(doc.page_content for doc in docs)

chain = (
    {"context":retriver | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)
chain.invoke("Co to jest AI tinkers??")

'AI Tinkers to seria spotkań lub wydarzeń związanych z sztuczną inteligencją, na których uczestnicy mogą dzielić się swoimi projektami, pomysłami i doświadczeniami w tej dziedzinie. Nie jest to jednak oficjalna nazwa ani organizacja; wydaje się, że jest to nieformalna inicjatywa lub cykl spotkań zainteresowanych osób.'